In [1]:
import os
import numpy as np

In [2]:
def ExtractFrames(file_path, pos=[0.1,0.3,0.5,0.7,0.9]):
    # Extracts frames from file_path at the positions (relative between 0 and 1) in pos
    
    import os
    
    if not len(pos):
        print("[ExtractFrames]: Invalid positions")
        return None
    
    if not os.path.isfile(file_path) :
        print("[ExtractFrames]: Invalid file path")
        return None
    
    import cv2
    
    # container for frames
    arr = np.empty((len(pos),224,224,3))
    
    cap = cv2.VideoCapture(file_path)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    for k,i in enumerate(pos):
        # get frame number
        position = int(i * total_frames)
        
        # set frame pointer at i and extract frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        
        # preprocessing
        frame = cv2.resize(frame, (224,224))
        frame = frame * 1/255.
        frame = np.float32(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # insert in container
        arr[k] = frame
        
    # cleanup
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    cap.release()
    
    return arr

In [11]:
def test_flow_from_directory(dir_path):
    import numpy as np
    import os
    from tqdm import tqdm
    
    # 2 classes: "brushing_teeth", "miscellaneous"
    posDIR = os.path.join(dir_path, "brushing_teeth")
    negDIR = os.path.join(dir_path, "miscellaneous")
    
    pos_samples = os.listdir(posDIR)
    neg_samples = os.listdir(negDIR)
    
    # Get all the videos in both classes
    pos_samples = [os.path.join(posDIR, fname) for fname in pos_samples]
    neg_samples = [os.path.join(negDIR, fname) for fname in neg_samples]
    
    sample_size = len(pos_samples) + len(neg_samples)
    
    # Array for data, labels and files 
    test_data   = np.empty( (sample_size, 5, 224, 224, 3) )
    test_label  = np.empty( (sample_size, 1) )
    test_files  = pos_samples+neg_samples
    
    # Extract frames from all videos using default positions
    index = 0
    print("Now extracting brushing_teeth videos")
    for vid in tqdm(pos_samples):
        test_data[index] = ExtractFrames(str(vid))
        test_label[index] = 1.
        index += 1
    
    print("Now extracting miscellaneous videos")
    for vid in tqdm(neg_samples):
        test_data[index] = ExtractFrames(vid)
        test_label[index] = 0.
        index += 1
    
    return test_data, test_label, test_files